# Programming On Linux




## 1 The software for programming C/C++ and  Python on Linux

### C/C++

* ❑ GCC: The GNU Compiler Collection, containing the GNU C compiler

* ❑ G++: A C++ compiler, included as part of GCC

* ❑ GNU make: A version of UNIX make

Most Linux distributions, such as Ubuntu, have built-in GCC compilers and do not require additional installation


If you use the Debian/Ubuntu distribution do not has built-in GCC compilers. you can install the folloing command
 
```bash 
$sudo apt install build-essential
```

The command installs a bunch of new packages including gcc, g++ and make.

### Python

All Linux distributions have built-in Python2 and Python2. However, it is incomplete

* Add 3 to the relevant commands of Python3


For Python3, we need to be supplemented with `pip3` and `idle3`.

```bash
$sudo apt install python3-pip
$sudo apt install idle3
```

**Install Python3 packages:**


* set `tsinghua` as the mirror site of Pypi

```bash
$sudo -H python3 -m pip config set global.index-url https://pypi.tuna.tsinghua.edu.cn/simple
```
* upgrade pip3

```bash
$sudo -H python3 -m  pip install pip -U
```
* Installing Packages

```bash
$sudo -H python3 -m pip install jupyter 
```

```bash
$sudo -H  python3 -m pip install coolprop 
```

### The text editor and programming environment

**Visual Studio Code**

From  https://code.visualstudio.com/docs/?dv=linux64_deb  download VS Code for Debian/Ubuntu,

then install in the terminal

```bash
$sudo dpkg -i [vscode filename].deb
```



### Version Control:Git

```bash 
$sudo apt install git
```


   
## 2 C/C++

In [1]:
%cd demo

J:\SEU\SEE\PySEE\home\notebook\demo


In [1]:
%pwd

'/mnt/j/SEU/SEE/PySEE/home/notebook'

### 2.1 Bisection Methond in C

𝑦=𝑓(𝑥) 

* ./include/roots.h

* ./src/roots.c

In [ ]:
# %load ./include/roots.h
#ifndef ROOTS_H
#define ROOTS_H

#include <math.h>
// function definition
typedef double (*fun)(double);

int rtbis(fun func,double y,double x1, double x2, double xacc,double *rtb);
/*
	The program uses the bisection method to solve the equation
		f(x)-y = 0.
	The solution is to be in [x1,x2] and it is assumed that
		(f(x1)-y)*(f(x2)-y) <= 0.
	The solution is returned in rtb, and it is to be in error by at most xacc.
	
	return value is an error indicator.
	  If =0, the solution has been computed satisfactorily.
	  If =1, (f(x1)-y)*(f(x2)-y) was greater than 0, contrary to assumption 
      If =2, exceeded the maximum number of iteration 
*/
#endif


In [ ]:
# %load ./src/roots.c
/*
  Numerical Recipes http://numerical.recipes
*/ 
#include <math.h>
#include "roots.h"

int rtbis(fun func,double y,double x1, double x2, double xacc,double *rtb)
{
 /*
	The program uses the bisection method to solve the equation
		f(x)-y = 0.
	The solution is to be in [x1,x2] and it is assumed that
		(f(x1)-y)*(f(x2)-y) <= 0.
	The solution is returned in rtb, and it is to be in error by at most xacc.
	
	return value is an error indicator.
	  If =0, the solution has been computed satisfactorily.
	  If =1, (f(x1)-y)*(f(x2)-y) was greater than 0, contrary to assumption 
      If =2, exceeded the maximum number of iteration 
*/
	const int IMAX=100; // the maximum number of iteration
    float dx,f,fmid,xmid;

	f=(*func)(x1)-y;
	fmid=(*func)(x2)-y;
	if (f*fmid >= 0.0) // endpoints do not straddle y=0
       return 1; 
    // init the root value: rtb
	*rtb = f < 0.0 ? (dx=x2-x1,x1) : (dx=x1-x2,x2);
	for (int i=1;i<=IMAX;i++) {
		fmid=(*func)(xmid=(*rtb)+(dx *= 0.5))-y;
		if (fmid <= 0.0) *rtb=xmid;
		if (fabs(dx) < xacc || fmid == 0.0) 
           return 0;
  	}
    // Exceeded the maximum number of iteration
    return 2;
}


### 2.2 The caller of Bisection Methond 

$𝑓(𝑥)=𝑥2$ 

$4=𝑓(𝑥)$

```c
int rtbis(fun func,double y,double x1, double x2, double xacc,double *rtb)

double xl, xu, epsilon, root;
ier=rtbis(fcn,y,xl, xu, epsilon, &root);
```
&root > double *rtb

In [ ]:
# %load ./src/rtbisApp.c

#include <stdio.h>
#include <string.h>
#include <stdlib.h>
#include "roots.h"

double fcn(double x)
{
	double result;
	result = x * x;
	return result;
}

int main()
{
	double xl, xu, epsilon, root;
	int ier;
	xl = 0.1;
	xu = 3.2;
	epsilon = 0.001;
    double y=4;
	// Calculate root
	ier=rtbis(fcn,y,xl, xu, epsilon, &root);
	// Print answers
	printf("root = %14.7e  ier = %1d", root, ier);
	return 0;
}


In [2]:
!gcc -o ./bin/rtbisApp ./src/rtbisApp.c  ./src/roots.c -I./include/

In [3]:
!./bin/rtbisApp

root =  1.9996581e+00  ier = 0

**Further Reading:**

**Path separators**

* Linux: /

* Windows: \

Escape character \

* The backslash`\` is an escape character,you use to inform that the next character is special.


### 2.3 The shared library

Linux supports two classes of libraries, namely:

* Static libraries – are bound to a program statically at compile time. 
* Shared libraries – are loaded when a program is launched and loaded into memory and binding occurs at run time. 

GCC assumes that all libraries on Linux

* start with `lib`

* Static libraries : end with `.a`

* Shared libraries: end with `.so`

There are mainly standard locations in the filesystem where a library can be placed.

* /lib, /lib64:   system startup libraries
* /usr/lib,/usr/lib64;
* /usr/local/lib, /usr/local/lib64





#### 2.3.1 creating the shared  library

using `__cdecl` calling convention

In [4]:
!gcc -c -O3 -Wall -fPIC -o ./obj/roots.o  ./src/roots.c -I./include
!gcc -shared -o ./bin/libroots.so  ./obj/roots.o

In [5]:
!ls ./bin/lib*.*

./bin/libfuns.dll  ./bin/libroots.so  ./bin/libstat.dll


### 2.3.2 Using the shared library

We use the same code `./src/statdemo.c` under Windows

**Linking the library**

**Windows**
```bash
    -L./bin/ -lstat
```
**Linux**
```bash
   -L./bin/ -lstat -Wl,-rpath=./bin/  
```
* **`-Wl,option`**

    Pass option as an option to the **linker**. 
    
    If option contains `commas`, it is split into multiple options at the commas. 
    
    You can use this syntax to pass an argument to the option.
      
   
* **`-rpath=dir`** 

   **Add a directory to the `runtime` library search path**. This is used when linking an executable with shared objects. 


In [6]:
!gcc -o  ./bin/rtbisApp ./src/rtbisApp.c -L./bin/ -lroots -I./include -Wl,-rpath=./bin/ 

In [7]:
!./bin/rtbisApp

root =  1.9996581e+00  ier = 0

## 3 Using The CoolProp Shared Library

**Change working dir to `./notebook/coolpropdemo`**

In [2]:
%cd coolpropdemo

/mnt/j/SEU/SEE/PySEE/home/notebook/coolpropdemo


In [3]:
%pwd

'/mnt/j/SEU/SEE/PySEE/home/notebook/coolpropdemo'

### Shared libraries Name on Linux


**soname**

Every shared library has a special name called the `soname`. The soname has the prefix `lib`, the name of the library, the phrase `.so` followed by a version number 

The soname is used to indicate what binary **api compatibility** your library support by providing version **backwards-compatibility** information. For instance, if version 6.0.0 through 6.4.1 of the shared library `libCoolProp` provide identical interfaces, they all have the same soname, `libCoolProp.so.6` 

On a working system. a fully-qualified soname is simply `a symbolic link` to the shared library's `real name`.

The symlink is what programs using the library will look for, the running programs do not care about the **version number.**


**real name**

Every shared library also has a `real name`, which is the filename containing the actual library code. The real name adds to the soname a  minor number and the release number. 

**linker name**

linker name is the name that the compiler uses when requesting a library, which is simply the soname **without any version number**

The linker name is simply created as `a symbolic link to the soname` 

The symlink is what programs using the library will look for, the running programs do not care about the **version number**

> **A symbolic link** is a `file` that links to another file or directory using its path. you can think of a symbolic link as `a shortcut` to a file or directory (folder). symbolic links may be used in the command line, or in a script or another program.

These naming conventions help **multiple** versions of same shared library to **co-exist** in a system.

* The programs linking with the shared library do not need to take care about **the latest version** of the shared library installed in the system. Once the latest version of the shared library is installed successfully, all the programs automatically start linking to the latest version.

The name used by linker is usually a symbolic link to the fully qualified soname which in turn is a symbolic link to the real name.

* linker name(a symbolic link) > soname(a symbolic link) > real name.

`libCoolProp.so.6.4.1`  is the library builded with GCC 9.3 and glibc2.29 above on Linux,

`libCoolProp.so.6.4.1` is **real name**

To find the soname of the library run:

In [2]:
%pwd

'/mnt/j/SEU/SEE/PySEE/home/notebook'

In [4]:
!readelf -a ./coolpropdemo/bin/libCoolProp.so.6.4.1 | grep SONAME

 0x000000000000000e (SONAME)             Library soname: [libCoolProp.so.6]


The `libCoolProp.6` is the soname. 

The `libCoolProp` is the linker name

type the following command in the terminal of `./coolpropdemo/` to create the symbolic link `libCoolProp.so.6` and `libCoolProp.so`

```bash
$sudo ln -sf libCoolProp.so.6.4.1 ./bin/libCoolProp.so.6
$sudo ln -sf libCoolProp.so.6 ./bin/libCoolProp.so
```
The soname `libCoolProp.so.6`  is a symlink which points to the real name(actual library) `libCoolProp.so.6.4.1`. 

The linker name `libCoolProp.so`  is a symlink which points to the soname `libCoolProp.so.6` 

In [12]:
%%file ./mainso.c

/* 
The example use the shared library of CoolProp in c

g++ -o ./bin/mainso  -DCOOLPROP_LIB mainso.c -I./include -L./bin -lCoolProp -Wl,-rpath=./bin/ 
       
./bin/mainso

*/

#include "CoolPropLib.h"
#include <stdio.h>

int main() {
     double value=PropsSI("P","T",273.15+0,"Q",0,"R134a");
     printf("%f\n", value); 
     return 0;
}

Overwriting ./mainso.c


In [10]:
!g++ -o ./bin/mainso  -DCOOLPROP_LIB mainso.c -I./include -L./bin -lCoolProp -Wl,-rpath=./bin/ 

In [11]:
!./bin/mainso

292803.182339


## 4 GNU Scientific Library(GSL)

GNU Scientific Library:  https://www.gnu.org/software/gsl/

The **GNU** Scientific Library (GSL) is a **numerical library** for C and C++ programmers. 

The library provides a wide range of mathematical routines such as random number generators, special functions and least-squares fitting. There are over 1000 functions in total with an extensive test suite.

The GSL is part of the GNU Project and is `free` software under the GNU General Public License.




### Using the GSL Library under Linux

#### 1 Compiling

The library header files are installed in their own `gsl` directory. You should write any preprocessor include statements with a `gsl/` directory prefix thus, 

```c
#include <gsl/gsl_statistics.h>
```
If the directory is not installed on the standard search path of your compiler you will also need to provide its location to the preprocessor as a command line flag.The default location of the gsl directory(ubuntu18.04) is `/usr/include/gsl`

In [2]:
%%file ./demo/src/gsl-example.c

#include <stdio.h>
#include <gsl/gsl_statistics.h>

int main(void)
{
  double data[5] = {17.2, 18.1, 16.5, 18.3, 12.6};
  double mean, variance, largest, smallest;

  mean     = gsl_stats_mean(data, 1, 5);
  variance = gsl_stats_variance(data, 1, 5);
  largest  = gsl_stats_max(data, 1, 5);
  smallest = gsl_stats_min(data, 1, 5);

  printf ("The dataset is %g, %g, %g, %g, %g\n",
         data[0], data[1], data[2], data[3], data[4]);

  printf ("The sample mean is %g\n", mean);
  printf ("The estimated variance is %g\n", variance);
  printf ("The largest value is %g\n", largest);
  printf ("The smallest value is %g\n", smallest);
  return 0;
}

Writing ./demo/src/gsl-example.c


A typical compilation command for a source file `gsl-example.c` with the GNU C compiler gcc is, 


In [8]:
!gcc -o ./demo/obj/gsl-example.o -c ./demo/src/gsl-example.c -I/usr/include 

This results in an object file `gsl-example.o`.

In [9]:
!ls ./demo/obj/gsl-example.*

./demo/obj/gsl-example.o


 The default include path for gcc searches `/usr/include` automatically so the `-I` option can actually be **omitted** when GSL is installed in its default location. 



In [10]:
!gcc -o ./demo/obj/gsl-example.o -c ./demo/src/gsl-example.c

In [11]:
!ls ./demo/obj/gsl-example.*

./demo/obj/gsl-example.o


### 2 Linking programs with the library

The library is installed as a single file, `libgsl.a`. A shared version of the library `libgsl.so` is also installed on systems that support shared libraries. 

The default location of these files(Ubuntu18.04) is `/usr/lib/x86_64-linux-gnu/`. If this directory is not on the standard search path of your linker you will also need to provide its location as a command line flag.

To link against the library you need to specify both the `math library` and a supporting` CBLAS library,` which provides standard basic linear algebra subroutines. A suitable `CBLAS` implementation is provided in the library `libgslcblas.a` if your system does not provide one. The following example shows how to link an application with the library:

The option `-lm` links with the system **math** library that has common math functions like sqrt, sin, cos, log, etc.
 
* **When you compile C codes with gcc, you neads adding `-lm`**


In [13]:
!gcc -o ./demo/bin/gsl-example  ./demo/obj/gsl-example.o -L/usr/lib/x86_64-linux-gnu/ -lgsl -lgslcblas -lm

In [14]:
!./demo/bin/gsl-example

The dataset is 17.2, 18.1, 16.5, 18.3, 12.6
The sample mean is 16.54
The estimated variance is 5.373
The largest value is 18.3
The smallest value is 12.6


The default library path for gcc searches `/usr/lib/x86_64-linux-gnu/` automatically so the `-L` option can be **omitted** when GSL is installed in its default location.

In [15]:
!gcc -o ./demo/bin/gsl-example  ./demo/obj/gsl-example.o  -lgsl -lgslcblas -lm

In [16]:
!./demo/bin/gsl-example

The dataset is 17.2, 18.1, 16.5, 18.3, 12.6
The sample mean is 16.54
The estimated variance is 5.373
The largest value is 18.3
The smallest value is 12.6


## Reference

https://github.com/PySEE/home/blob/B2021/guide/doc/Ubuntu-Python-CPP(Chinese).md